In [41]:
import telegram 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import io 
import pandas as pd
import pandahouse
from datetime import datetime, timedelta

In [42]:
token = '7044101108:AAETY0C2dU2iAJAV-N7y497ztoXxCBloq2E'
bot = telegram.Bot(token=token)
chat_id = 1027001565

In [43]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}

In [44]:
def check_anomaly_iqr(df_metric, metric, a=3, n=5):
    # функция предлогает алгоритм поиска аномалий в данных (межквартильный размах)
    df_metric['q25'] = df_metric[metric].shift(1).rolling(n).quantile(0.25)
    df_metric['q75'] = df_metric[metric].shift(1).rolling(n).quantile(0.75)
    df_metric['iqr'] = df_metric['q75'] - df_metric['q25']
    df_metric['up'] = df_metric['q75'] + a * df_metric['iqr']
    df_metric['low'] = df_metric['q25'] - a * df_metric['iqr']
    
    df_metric['up'] = df_metric['up'].rolling(n, center=True, min_periods=1).mean()
    df_metric['low'] = df_metric['low'].rolling(n, center=True, min_periods=1).mean()
    
    if df_metric[metric].iloc[-1] < df_metric['low'].iloc[-1] or df_metric[metric].iloc[-1] > df_metric['up'].iloc[-1]:
        is_alert = 1
    else:
        is_alert = 0
    return is_alert, df_metric

In [45]:
def ren_alerts(df):
    # система алертов
    metrics_list = list(df.columns[3:])
    for metric in metrics_list:
        df_metric = df[['ts', 'date', 'hm', metric]].copy()
        is_alert, df_metric = check_anomaly_iqr(df_metric, metric)
        
        if is_alert == 1 or True:
            msg = '''
            Метрика {metric}.\nТекущее значение {current_val:.2f}. Отклонение более {last_val:.2%}.\nСсылка: http://superset.lab.karpov.courses/r/5717\n@zhaskanatshaimurat
            '''.format(metric=metric, current_val=df_metric[metric].iloc[-1], last_val=1-df_metric[metric].iloc[-1]/df_metric[metric].iloc[-2])
            
            
            sns.set(rc={'figure.figsize': (16, 10)})
            plt.tight_layout()
            
            ax = sns.lineplot(x=df_metric['ts'], y=df_metric[metric], label='metric')
            ax = sns.lineplot(x=df_metric['ts'], y=df_metric['up'], label='up')
            ax = sns.lineplot(x=df_metric['ts'], y=df_metric['low'], label='low')
            
            for ind, label in enumerate(ax.get_xticklabels()):
                if ind % 3 == 0:
                    label.set_visible(True)
                else:
                    label.set_visible(False)
            
            ax.set(xlabel='time')
            ax.set(ylabel=metric)
            
            ax.set_title(metric)
            ax.set(ylim=(0, None))
            
            plot_object = io.BytesIO()
            plt.savefig(plot_object)
            plot_object.seek(0)
            plot_object.name = '{0}.png'.format(metric)
            plt.close()
            
            bot.sendMessage(chat_id=chat_id, text=msg)
            bot.sendPhoto(chat_id=chat_id, photo=plot_object)
            
    return 

In [46]:
    query = '''
        SELECT toStartOfFifteenMinutes(time) as ts,
               toDate(time) as date,
               formatDateTime(ts, '%R') as hm,
               uniqExact(user_id) as users_feed,
               countIf(user_id, action='view') as views,
               countIf(user_id, action='like') as likes,
               countIf(user_id, action='like')/countIf(user_id, action='view') as CTR
        FROM simulator_20240620.feed_actions
        WHERE time >= today() - 1 and time < toStartOfFifteenMinutes(now())
        GROUP BY ts, date, hm
        ORDER BY ts
    '''

    df_feed_actions = pandahouse.read_clickhouse(connection=connection, query=query)

In [47]:
    query = '''
        SELECT toStartOfFifteenMinutes(time) as ts,
               toDate(time) as date,
               formatDateTime(ts, '%R') as hm,
               uniqExact(user_id) as users_message,
               count(user_id) as messages
        FROM simulator_20240620.message_actions
        WHERE time >= today() - 1 and time < toStartOfFifteenMinutes(now())
        GROUP BY ts, date, hm
        ORDER BY ts
    '''

    df_message_actions = pandahouse.read_clickhouse(connection=connection, query=query)

In [49]:
ren_alerts(df_feed_actions)

In [50]:
metric_feed_actions = list(df_feed_actions.columns[3:])

In [51]:
metric_feed_actions

['users_feed', 'views', 'likes', 'CTR']

In [40]:
metric_message_actions

NameError: name 'metric_message_actions' is not defined

In [25]:
superset_url = 'http://superset.lab.karpov.courses/r/5717'